In [9]:
# Ajoutez cette cellule au début de votre notebook pour corriger l'erreur matplotlib
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pygame
import sys
from Player import Player  # Import Player from external file
from CircuitSimulator import CircuitSimulator
from GameUI import GameUI
import numpy as np

# --- Game Setup ---

pygame.init()
WIDTH, HEIGHT = 1500, 600  
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Quantum Gate Minigame")
font = pygame.font.SysFont(None, 32)
players = [Player("Mario"), Player("Luigi"), Player("Yoshi"), Player("Peach")]
current_player = 0
skipped_players = set()

# --- Gate UI Setup ---
GATE_COLORS = {"H": (200,200,255),"Z": (255,200,200),"Y": (200,255,200),"X": (255,255,200),"CNOT": (200,255,255),"SWAP": (255,200,255), "DECOH": (120,120,120)}
GATE_LIST = ["H","Z","Y","X","CNOT","SWAP"]
gate_rects = {}
for i, gate in enumerate(GATE_LIST):
    gate_rects[gate] = pygame.Rect(30, 50 + i*60, 80, 40)

# --- Circuit State ---
MAX_GATES = 20
qiskit_circuit = CircuitSimulator.create_empty_circuit()
gate_history = [("H", 0), ("H", 1, "layer0")]  

# --- Decoherence State ---
def get_decoherence_percent():
    # Decoherence is now 10% per DECOH gate (max 100)
    decoh_gates = [g for g in gate_history if g[0] == "DECOH"]
    percent = min(100, len(decoh_gates) * 20)
    return percent

# --- Drag and Drop ---
dragging_gate = None
drag_offset = (0,0)
drag_pos = (0,0)

# --- Main Loop ---

measurement_result = None
measurement_probs = None  # Ajout pour stocker les probabilités

def next_player():
    global current_player
    n = len(players)
    for _ in range(n):
        current_player = (current_player + 1) % n
        if current_player not in skipped_players:
            return

def get_probabilities(qc, gate_history=None):
    # Ignore decoherence for probability calculation
    from qiskit import QuantumCircuit
    from qiskit_aer import AerSimulator
    if gate_history is not None:
        # Rebuild circuit from history (no decoherence)
        qc_copy = QuantumCircuit(2, 2)
        for g in gate_history:
            if g[0] == "H":
                qc_copy.h(g[1])
            elif g[0] == "X":
                qc_copy.x(g[1])
            elif g[0] == "Y":
                qc_copy.y(g[1])
            elif g[0] == "Z":
                qc_copy.z(g[1])
            elif g[0] == "CNOT":
                qc_copy.cx(g[1], g[2])
            elif g[0] == "SWAP":
                qc_copy.swap(g[1], g[2])
            # Skip "DECOH"
        qc_copy.measure(0, 0)
        qc_copy.measure(1, 1)
    else:
        try:
            qc_copy = qc.copy()
        except AttributeError:
            qc_copy = QuantumCircuit.from_qasm_str(qc.qasm())
        has_measure = any(instr[0].name == "measure" for instr in qc_copy.data)
        if not has_measure:
            qc_copy.measure(0, 0)
            qc_copy.measure(1, 1)
    sim = AerSimulator()
    result = sim.run(qc_copy, shots=32768).result()
    counts = result.get_counts()
    total = sum(counts.values())
    probs = {k: v / total for k, v in counts.items()}
    for state in ["00", "01", "10", "11"]:
        if state not in probs:
            probs[state] = 0.0
    return probs

def draw_probability_table(probs):
    # Dessine un graphique à barres comme dans IBM composer
    import matplotlib.pyplot as plt
    import matplotlib
    matplotlib.use("Agg")  # Utilisation du backend non interactif pour éviter les crashs pygame/matplotlib
    states = ["00", "01", "10", "11"]
    values = [probs.get(s, 0) for s in states]
    fig, ax = plt.subplots(figsize=(5,2.5))
    bars = ax.bar(states, values, color="#64b5f6", width=0.5)
    ax.set_ylim(0, 1)
    ax.set_ylabel("Probability")
    ax.set_xlabel("State")
    for bar, v in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f"{int(v*100)}%", ha='center', color='black', fontsize=12)
    plt.close(fig)  # Ferme la figure pour éviter les conflits avec pygame

def draw_probability_table_pygame(probs, screen, font, width, height):
    # Affiche le graphe de probabilité directement dans la fenêtre pygame
    bar_width = 50
    bar_gap = 30
    base_x = width // 2 - (2 * (bar_width + bar_gap))
    base_y = height - 250
    max_height = 120
    # Axes
    pygame.draw.line(screen, (200,200,200), (base_x-20, base_y), (base_x+4*(bar_width+bar_gap), base_y), 2)
    pygame.draw.line(screen, (200,200,200), (base_x-20, base_y), (base_x-20, base_y-max_height-10), 2)
    # Barres
    for i, state in enumerate(["00", "01", "10", "11"]):
        prob = probs.get(state, 0)
        bar_h = int(prob * max_height)
        x = base_x + i * (bar_width + bar_gap)
        y = base_y - bar_h
        pygame.draw.rect(screen, (100,180,255), (x, y, bar_width, bar_h))
        # Pourcentage
        pct_txt = font.render(f"{int(prob*100)}%", True, (0,0,0))
        screen.blit(pct_txt, (x+5, y-25))
        # Label état
        label_txt = font.render(state, True, (0,0,0))
        screen.blit(label_txt, (x+10, base_y+10))

while True:
    screen.fill((240,240,240))
    GameUI.draw_player_info(screen, font, players, current_player)
    GameUI.draw_gates(screen, font, players[current_player], players, current_player, gate_rects, GATE_LIST, GATE_COLORS)
    GameUI.draw_circuit(screen, font, gate_history, GATE_COLORS, MAX_GATES)
    btn_rect = GameUI.draw_measure_button(screen, font, WIDTH, HEIGHT)
    skip_btn_rect = GameUI.draw_skip_button(screen, font, WIDTH, HEIGHT)
    # Draw decoherence percent
    
    decoh_percent = get_decoherence_percent()

    decoh_txt = font.render(f"Decoherence chance: {decoh_percent}%", True, (120,0,0))
    screen.blit(decoh_txt, (WIDTH-350, HEIGHT-400))
    if measurement_result:
        txt = font.render(f"Measured: {measurement_result}", True, (0,0,0))
        screen.blit(txt, (WIDTH-300, HEIGHT-340))
    # Affiche le graphe de probabilité si disponible
    if measurement_probs:
        draw_probability_table_pygame(measurement_probs, screen, font, WIDTH, HEIGHT)
    # Draw dragging gate if any
    if dragging_gate:
        mx, my = drag_pos
        pygame.draw.rect(screen, GATE_COLORS[dragging_gate], (mx-40, my-20, 80, 40))
        txt = font.render(dragging_gate, True, (0,0,0))
        screen.blit(txt, (mx-20, my-10))
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos
            if skip_btn_rect.collidepoint(mx, my):
                skipped_players.add(current_player)
                next_player()
                dragging_gate = None
                drag_pos = (0,0)
                continue
            for gate, rect in gate_rects.items():
                if rect.collidepoint(mx, my) and players[current_player].gates[gate] > 0:
                    dragging_gate = gate
                    drag_offset = (mx - rect.x, my - rect.y)
                    drag_pos = (mx, my)
            if btn_rect.collidepoint(mx, my):
                # Apply decoherence noise model
                percent = get_decoherence_percent()
                noise_model = CircuitSimulator.apply_decoherence_noise(qiskit_circuit, percent)
                measurement_result = CircuitSimulator.apply_circuit(qiskit_circuit, noise_model=noise_model,gate_history=gate_history)
                # Met à jour le graphe de probabilité lors de la mesure
                measurement_probs = get_probabilities(qiskit_circuit, gate_history=gate_history)
        elif event.type == pygame.MOUSEBUTTONUP:
            if dragging_gate:
                mx, my = event.pos
                base_x = 200
                gate_layer = max(0, len(gate_history) - 2)
                drop_x = base_x + (gate_layer+2)*60
                dropped = False
                for q in range(2):
                    y = 150 + q*60
                    if drop_x-20 < mx < drop_x+20 and y-20 < my < y+20:
                        gate_count = players[current_player].gates.get(dragging_gate)
                        if gate_count is not None and gate_count > 0:
                            try:
                                if dragging_gate == "CNOT":
                                    control = q
                                    target = 1 - q
                                    qiskit_circuit.cx(control, target)
                                    gate_history.append(("CNOT", control, target))
                                    players[current_player].gates["CNOT"] = max(0, players[current_player].gates.get("CNOT", 0) - 1)
                                elif dragging_gate == "SWAP":
                                    qiskit_circuit.swap(0, 1)
                                    gate_history.append(("SWAP", 0, 1))
                                    players[current_player].gates["SWAP"] = max(0, players[current_player].gates.get("SWAP", 0) - 1)
                                else:
                                    if dragging_gate == "H":
                                        qiskit_circuit.h(q)
                                    elif dragging_gate == "X":
                                        qiskit_circuit.x(q)
                                    elif dragging_gate == "Y":
                                        qiskit_circuit.y(q)
                                    elif dragging_gate == "Z":
                                        qiskit_circuit.z(q)
                                    gate_history.append((dragging_gate, q))
                                    players[current_player].gates[dragging_gate] = max(0, players[current_player].gates.get(dragging_gate, 0) - 1)
                                next_player()
                                measurement_result = None
                                dropped = True
                                break
                            except Exception as e:
                                print("Erreur lors de l'ajout de la porte :", e)
                # Insert decoherence gate every 4 user-placed gates
                placed_gates = [g for g in gate_history[2:] if g[0] != "DECOH"]
                if len(placed_gates) > 0 and len(placed_gates) % 4 == 0:
                     gate_history.append(("DECOH", None))
                # Met à jour le graphe de probabilité à chaque placement de porte
                measurement_probs = get_probabilities(qiskit_circuit, gate_history=gate_history)
                dragging_gate = None
                drag_pos = (0,0)
        elif event.type == pygame.MOUSEMOTION:
            if dragging_gate:
                drag_pos = event.pos

SystemExit: 

c:\Users\gable\anaconda3\envs\IQuCodeFest\Lib\site-packages\IPython\core\interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
